In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk

In [2]:
df=pd.read_csv('/content/drive/MyDrive/tweets.csv')

In [3]:
df.head(5)

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [4]:
df=df[['text','sentiment']]

In [5]:
df.head(5)

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [6]:
df.shape

(13871, 2)

In [7]:
df=df[df.sentiment!='Neutral']


In [8]:
for i in range(len(df)):
  if df['sentiment'].iloc[i]=='Positive':  # Compare the value at index i
    df['sentiment'].iloc[i]=1
  else:
    df['sentiment'].iloc[i]=0

<ipython-input-8-a8658339c336>:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['sentiment'].iloc[i]=1
<ipython-input-8-a8658339c336>:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values th

In [9]:
df['sentiment'] = df['sentiment'].astype(int)


In [10]:
df.head(5)

,text,sentiment
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,1
3,RT @RobGeorge: That Carly Fiorina is trending ...,1
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,1
5,"RT @GregAbbott_TX: @TedCruz: ""On my first day ...",1
6,RT @warriorwoman91: I liked her and was happy ...,0


In [11]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [12]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [13]:
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')

In [14]:
corpus=[]
import re
for i in range (0,len(df)):
  review = str(df['text'].iloc[i])
  review=re.sub('[^a-zA-Z]',' ',review)
  review=review.lower()
  review=review.split()
  review= [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review=' '.join(review)
  corpus.append(review)





In [15]:
corpus[:5]

['rt scottwalk catch full gopdeb last night scott best line second walker http co zsff',
 'rt robgeorg carli fiorina trend hour debat men complet gopdeb say',
 'rt danscavino gopdeb w realdonaldtrump deliv highest rate histori presidenti debat trump http co',
 'rt gregabbott tx tedcruz first day rescind everi illeg execut action taken barack obama gopdeb foxnew',
 'rt warriorwoman like happi heard go moder not anymor gopdeb megynkelli http']

In [16]:
#bag of words
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=3000)
X=cv.fit_transform(corpus).toarray()
y=df['sentiment'].values

In [17]:
#split data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

Random forest model

In [18]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=10,criterion='entropy',random_state=42)
model.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10, random_state=42)

In [19]:
y_pred=model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8434296365330848

Naive Bayes model

In [21]:
from sklearn.naive_bayes import MultinomialNB
model2=MultinomialNB()
model2.fit(X_train,y_train)

MultinomialNB()

In [22]:
y_pred2=model2.predict(X_test)

In [23]:
accuracy_score(y_test,y_pred2)

0.8392357875116496

test random forest model

In [24]:
single_review="Godfather movie is not good"
review=re.sub('[^a-zA-Z]',' ',single_review)
review=review.lower()
review=review.split()
review= [ps.stem(word) for word in review if not word in set(all_stopwords)]
review=' '.join(review)
pros_rev=cv.transform([review]).toarray()
result=model.predict(pros_rev)
if result ==1:
  print("Positive Review")
else:
  print("Negative Review")

Negative Review


NN model (LSTM)

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np


X_train_reshaped = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])


X_train_reshaped = X_train_reshaped.astype('float32')
print("X_train_reshaped shape:", X_train_reshaped.shape)
print("y_train shape:", y_train.shape)
# Define the LSTM model
model = Sequential([
    Input(shape=(1, 3000)),
    LSTM(units=64, return_sequences=False),
    Dropout(0.2),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='sigmoid')
])


X_train_reshaped shape: (8583, 1, 3000)
y_train shape: (8583,)


In [26]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Train the model
model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

Epoch 1/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8000 - loss: 0.5212 - val_accuracy: 0.8526 - val_loss: 0.3524
Epoch 2/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.8829 - loss: 0.2928 - val_accuracy: 0.8602 - val_loss: 0.3393
Epoch 3/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - accuracy: 0.9077 - loss: 0.2296 - val_accuracy: 0.8492 - val_loss: 0.3801
Epoch 4/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9285 - loss: 0.1873 - val_accuracy: 0.8474 - val_loss: 0.4227
Epoch 5/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.9365 - loss: 0.1557 - val_accuracy: 0.8346 - val_loss: 0.4523
Epoch 6/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.9477 - loss: 0.1385 - val_accuracy: 0.8393 - val_loss: 0.5237
Epoch 7/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.9537 - loss: 0.1192 - val_accuracy: 0.8334 - val_loss: 0.5750
Epoch 8/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9566 - loss: 0.1062 - val_accu

In [28]:
y_train_pred = (model.predict(X_train_reshaped) > 0.5).astype(int)
train_accuracy_score = accuracy_score(y_train, y_train_pred)

269/269 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


Accuracy for LSTM

In [29]:
train_accuracy_score

0.9406967260864499

Test LSTM

In [30]:
single_review = "interstaller is great movie ever"

# Preprocess the review
review = re.sub('[^a-zA-Z]', ' ', single_review)
review = review.lower()
review = review.split()
review = [ps.stem(word) for word in review if word not in all_stopwords]
review = ' '.join(review)
print("Processed review:", review)

pros_rev = cv.transform([review]).toarray()

pros_rev = pros_rev.reshape(pros_rev.shape[0], 1, pros_rev.shape[1])

# Predict using the LSTM model
result = model.predict(pros_rev)

if result > 0.5:
    print("Positive Review")
else:
    print("Negative Review")

Processed review: interstal great movi ever
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 197ms/step
Positive Review
